In [153]:
import pandas as pd
import nltk
import csv

from nltk.corpus import stopwords
stop_words = set(stopwords.words("english"))
from nltk.tokenize import RegexpTokenizer
from nltk.stem.snowball import SnowballStemmer
#from googletrans import Translator

stemmer = SnowballStemmer('english')
#translator = Translator()
from pathlib import Path
import math
import numpy as np
import re
from scipy import spatial


from IPython.display import HTML, display


In [10]:
#nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml


True

In [61]:
# Reading tsvs and getting cleaned stemmed dictionary

In [5]:
#load the given dataset into pandas for processing using only the needed columns
data = pd.read_csv('Airbnb_Texas_Rentals.csv', usecols= ['average_rate_per_night', 'bedrooms_count','city', 'date_of_listing', 'description', 'latitude','longitude','title', 'url'])


In [7]:
#create a file for every row in data(the given data of airbnb)
fileCount = 0
for index, r in data.iterrows():
    data_temp = data.loc[index:index]
    fileCount +=1
    data_temp.to_csv('data/doc_'+str(index+1)+'.tsv', sep='\t', index=False, header=False)

In [8]:
# Organizing the functions


    # list of unique words (terms) to be used to build the vocabulary
words = []

def cleanData(rawData):
        tokenizer = RegexpTokenizer(r'\w+')
        stop_words = set(stopwords.words("english"))

        # TODO translatoin
        #lang = detect(t4[0])
        #print(lang)
        #rawData = translator.translate(rawData, dest='en')

        # get words lowercased
        t0 = rawData.lower()
        # remove puctuations
        t1 = tokenizer.tokenize(t0)
        #t2Data = stemmer.stem(i for i in t1Data)
        #print(len(t1))
        # reomve stop words
        t2 =[]
        t2 = [t1[i] for i in range(0,len(t1)) if t1[i] not in stop_words]

        # stemm words
        t3 = [stemmer.stem(t2[i]) for i in range(0, len(t2))]

        # remove nummbers and strings starting with numbers
        t4 = [t3[i] for i in range(0, len(t3)) if t3[i][0].isdigit()==False]


        #print(t4)
        return(t4)

def getWordsTSV(fid):
        fileWords=[]
        for i in open('data/doc_'+str(fid)+'.tsv', encoding="utf8"):
            #words.append(cleanData(i[0][4]))
            data = [i.strip('\n').split('\t') for i in open('data/doc_'+str(fid)+'.tsv',encoding="utf8")]
            for w in (cleanData(data[0][4].replace('\\n', ' '))):
                if w not in fileWords: fileWords.append(w)
        return(fileWords)


In [9]:

for i in range(1,fileCount):
    t = getWordsTSV(i)
    for w in t:
        if w not in words:
            words.append(w)
#print(words)

#### PS
dunno why the stemmer is removing the last 'e' from any word
we need to look at the stem function again later

In [11]:
# writing the vocabulary file from the words 
wordsCount=0
with open('vocabulary.csv','wb') as vfile:
    for i in range(0,len(words)):
        vfile.write(str(wordsCount).encode())
        vfile.write(str('\t').encode())
        vfile.write(str(words[i]).encode())
        vfile.write('\n'.encode())
        wordsCount+=1

In [12]:
print(wordsCount)

9617


In [ ]:
# DEPRECATED
'''# creating index-file by loading vocabulary file and then comparing files with all vocabularys
with open('vocabulary.csv', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    voc = list(map(tuple, reader))
#print(voc)

index = {}
for i in range(len(voc)):
    tempL=[]

    for j in range(1,fileCount+1):
        temp = getWordsTSV(j)
        print(i)
        if voc[i][1] in temp:
            tempL.append(j)
            
        if len(tempL)!=0: index[i]= tempL
        
            
#print(index)  '''

In [13]:
# creating index-file by loading vocabulary file and then comparing files with all vocabularys
with open('vocabulary.csv', newline='') as file:
    reader = csv.reader(file, delimiter='\t')
    voc = np.array(list(map(tuple, reader)))


In [14]:
print(voc[:10])

[['0' 'welcom']
 ['1' 'stay']
 ['2' 'privat']
 ['3' 'room']
 ['4' 'queen']
 ['5' 'bed']
 ['6' 'detach']
 ['7' 'bathroom']
 ['8' 'second']
 ['9' 'floor']]


In [57]:
vocWords=voc[:,1]

In [59]:
print(len(vocWords))

9617


In [125]:
# function to get ID of a given term (this one uses the voc array instead of reading from file)
def getIDfromVoc(term):
    for t in range(len(voc)):
        if voc[t,1] == term:
            return(t)
    return(-1)
# function to get Term for a given term ID
def getTermfromVoc(tid):
    for t in range(len(voc)):
        if voc[t,0] == tid:
            return(voc[t,1])
    return(None)

In [134]:
voc[0,1]
#print(getTermfromVoc(1))

'welcom'

In [16]:
# unit test
getIDfromVoc('welcom')

0

In [ ]:
# unit test
print(len())

In [17]:
# creating the index
index = {}
for f in range(1,fileCount+1):
    temp = getWordsTSV(f)
    for t in temp:
        termID=getIDfromVoc(t)
        if termID in index:
            index[termID].append(f)
        else:
            index[termID]=[f]
                

            
#print(index)  

In [ ]:
# unit test
print(index[36])

In [19]:
# we have done another fucntion with same functionality: find above!
def getID(term):
    with open('vocabulary.csv', newline='') as voc:
        line = csv.reader(voc, delimiter='\t')
        w = list(map(tuple, line))
        #print(w[0][1])
        for i in w:
            if term == i[1]:
                return (i[0])
        else:
            return None 



In [25]:
# NO NEED FOR THIS FUNCTION function takes term ID and returns (from index) the list of files containing the term
def getDocsWithID(tid):
    for k in index:
        if tid == k:
            return(index.get(k))
    return None

#getting error: TypeError: 'int' object is not iterable

In [26]:
# fixed it to check if the doc contains all terms
def getDocswithQuery(q):
    ts = cleanData(q)
    #docsIDs = getDocsWithID(int(getID(ts[0])))
    docsIDs = index[getIDfromVoc(ts[0])]
    if docsIDs == None : return None
    if len(ts)>1:
        #docsIDs = getDocsWithID(int(getID(ts[0])))
        for t in ts[1:]:
            tid = getIDfromVoc(t)
            if tid != -1:
                tempDocs = index.get(tid)
                if tempDocs != None:
                    docsIDs = set(docsIDs).intersection(tempDocs)
            else: return None
    return(list(docsIDs))

In [87]:
# deprecated, find below!
def old_getDetails(fid):
    with open('data/doc_'+str(fid)+'.tsv', encoding="utf8") as file:
        line = csv.reader(file, delimiter='\t')
        cls = list(map(str, *line))
        print(cls[7],cls[4], cls[2], cls[8])
        # TODO create a table and list the details


In [21]:
# funciton retuns the details of a file given file id

def getDetails(fid):
    detlist =[]
    with open('data/doc_'+str(fid)+'.tsv', encoding="utf8") as file:
        line = csv.reader(file, delimiter='\t')
        cls = list(map(str, *line))
        detlist= [cls[7],cls[4].rstrip('\r\n').replace('\\n', ' '), cls[2], cls[8]]
        #TODO create a table and list the details
        return detlist

In [22]:
import html
from bs4 import BeautifulSoup
def cleanhtml(raw_html):
    soup = BeautifulSoup(raw_html)
    for span_tag in soup.findAll('span'):
        span_tag.replace_with('')
    return(html.unescape(str(soup)))

In [27]:
# I defined two functions for the visualizing stuff
def hoohleResult(r):
    rhtm ='<div class="h_result"><div class="h_title"><a href="'+r[3]+'">'+r[0]+'</a></div>'
    rhtm +='<div class="h_link"><a href="'+r[3]+'">'+r[3]+'</a></div>'
    rhtm +='<div class="h_disc">'+r[1].replace('\n', '')[:300]+'... </div>'
    rhtm +='</div>'
    return (rhtm)
    
def hoohleResults(query):
    docs = getDocswithQuery(query)
    styles = open("style/style.css", "r").read()
    hhtm = '<style>%s</style>' % styles     
    hhtm +='<div class="Hoohle" ><div class="h_results">'
    if docs != None:
        for doc in docs:
            hhtm +=hoohleResult(getDetails(doc))
    else:
        hhtm +='Sorry! No results found for your request!'
    hhtm += '</div></div>'
    display(HTML(hhtm))


def printResults(query):
    docs = getDocswithQuery(query)
    #det =[['Title','Description', 'City','Url']]
    #for doc in docs:
    #    det.append(getDetails(doc))
    #columns = ('Title','Description', 'City','Url')
    #n_rows = len(det)
    html='<table align="left"><tr><th>Title</th><th>Description</th><th>City</th><th>URL</th></tr>'
    if docs != None:

        for doc in docs:
            l = getDetails(doc)
            html+='<tr><td>'+l[0]+'</td><td>'+l[1]+'</td><td>'+l[2]+'</td><td>'+l[3]+'</td></tr>'
    else:
        html+='<tr><td>Sorry! Nothing found!</td></tr>'
    html+='</table>'
    display(HTML(html))
    #table = ff.create_table(det, height_constant=60)
    
    #plotly.offline.iplot(table, filename='simple_table')
    #plt.show()
    #return det[0][1]
    
    '''
    maybe remove the stuff we commented out if we don't need it anymore?
    '''


In [28]:
hoohleResults("Welcome to stay i room with queen bed")

In [29]:
getDocswithQuery("Welcome to stay in private room with queen bed")

[9000, 1, 430]

9617


2110

In [63]:
#create dictionary with term frequency per fid by using the modified getWordsTSV function below

def getTF(fid, term):
    tfd = {}
    words = getAllWordsTSV(fid)
    n = 0
    for w in words:
        if w == term:
            n+1
    return (n/len(words))

def getIDF(word):
    return (math.log(fileCount/len(index(getIDfromDov(word)))))


    #get the ids of the documents in which the term occures
    files = getDocsWithID(int(getID(word)))

    #number of docs in which the terms occurs it the length of files
    #filecount = total number of files
    idf = math.log(fileCount/len(files))
    return idf

    if term in words:
        for word in words:
            if word == term:
                if word in tfd:
                    tfd[term] =  tfd[term]+1
                    #maybe replace word with term id from vocabulary
                else:
                    tfd[term] = 1
    else: return None
    for tfs in tfd:
        tfd[tfs] = tfd[tfs]/len(words)
    return tfd


#modified getWordsTSV function that creates a list with all words in the doc and does not remove the duplicates
def getWordsTSV_dup(fid):
    fileWords=[]
    for i in open('data/doc_'+str(fid)+'.tsv', encoding="utf8"):
        #words.append(cleanData(i[0][4]))
        data = [i.strip('\n').split('\t') for i in open('data/doc_'+str(fid)+'.tsv',encoding="utf8")]
        for w in (cleanData(data[0][4].replace('\\n', ' '))):
            fileWords.append(w)
    return(fileWords)
    

In [149]:
#calculate the inverse data frquency(idf) of a word
def getIDF(word):
    
    #get the ids of the documents in which the term occures
    files = getDocsWithID(int(getID(word)))

    #number of docs in which the terms occurs it the length of files
    #filecount = total number of files
    idf = math.log(fileCount/len(files))
    return idf
    
    

In [151]:
#calculate the tfidf for one word

# return all clean terms in a given file (including duplicates)
def getAllWordsTSV(fid):
    fileWords=[]
    for i in open('data/doc_'+str(fid)+'.tsv', encoding="utf8"):
        data = [i.strip('\n').split('\t') for i in open('data/doc_'+str(fid)+'.tsv',encoding="utf8")]
        for w in (cleanData(data[0][4].replace('\\n', ' '))):
            fileWords.append(w)
    return(fileWords)

# for a given (file id, term), returns the term frequency in this file
def get_TF(fid, term):
    words = getAllWordsTSV(fid)
    n = 0
    for w in words:
        if w == term:n+=1
    return (n/len(words))

# returns the IDF for a given term in the whole data we have
def get_IDF(word):
    return (math.log(fileCount/len(index[getIDfromVoc(word)])))
# returns the TFIDF for a given (file id, term)
def get_TFIDF(fid, termID):
    #getting TF for term in document with document id = fid and multiply it with the IDF of the term
    term = voc[termID][1]
    return ([fid, get_TF(fid, term) * get_IDF(term)])

In [151]:
getTFIDF(1,'bed')

0.1039563534909512

In [149]:
get_TFIDF(1,1)
#voc[1][1]

[1, 0.07707036291666503]

In [ ]:
#getAllWordsTSV(1)
#len(index[getIDfromVoc('bed')])

In [152]:
indexTFIDF = {}
for termID in index:
    indexTFIDF[termID]=[]
    for docID in index[termID]:
        indexTFIDF[termID].append(get_TFIDF(docID, termID))

In [120]:
getIDfromVoc(getTermfromVoc(1))

-1

In [130]:
print(getTermfromVoc(1))

None


In [163]:
indexTFIDF[1][:10]

[[1, 0.07707036291666503],
 [14, 0.05018535259689816],
 [23, 0.08631880646666483],
 [27, 0.11357737692982213],
 [30, 0.16599770474358622],
 [31, 0.041499426185896554],
 [40, 0.08299885237179311],
 [47, 0.05018535259689816],
 [56, 0.04071641814465322],
 [58, 0.03923582112121128]]

In [211]:
# function to get the cosine similarity between two given vectors
# TODO i got error when the file doesnt contain any term from the query!

def getCosineSimi(v1,v2):
    return (1 - spatial.distance.cosine(v1, v2))

In [160]:
# function to get the tfidf for a given query
def getTFIDF_query(q):
    terms = cleanData(q)
    v = []
    uniqTerms=[]
    for t in terms:
        if t not in uniqTerms:
            uniqTerms.append(t)
            v.append(terms.count(t))
    return([x/len(terms) for x in v])

In [230]:
# function to get the tfidf vector for a given query in a document 
def getTFIDF_vector(doc,terms):
    v=[]
    for term in list(set(cleanData(terms))):
        if term not in getWordsTSV(doc): v.append(0)
        else:
            l = indexTFIDF[getIDfromVoc(term)]
            for i in l:
                if i[0]==doc:
                    v.append(i[1])
    return(v)

In [224]:
qq = "I want a beautiful welcome beautiful house"
getCosineSimi(getTFIDF_query(qq), getTFIDF_vector(1,qq))

0.3779644730092272

In [226]:
getTFIDF_query(qq)

[0.2, 0.4, 0.2, 0.2]

In [231]:
getTFIDF_vector(1,qq)

welcom
want
hous
beauti


[0.10075844646484013, 0, 0, 0]

In [164]:
getTFIDF_query('hello hello fucking bad ass')

[0.4, 0.2, 0.2, 0.2]

In [ ]:
getCosineSimi(getTFIDF_query('hello hello fucking ass'),getTFIDF_query('hello hello fucking bad ass'))

In [207]:
getTFIDF_vector(1,'bed private queen')

[0.1039563534909512, 0.15827376478966393, 0.07700272220380536]

In [ ]:
indexTFIDF[getIDfromVoc('bed')]
#indexTFIDF[0][0][1]

In [ ]:
#BONUS QUESTION, HAVING RESULTS ON MAP

In [9]:
!pip install geopy
from geopy import distance

  Running setup.py bdist_wheel for geographiclib: started
  Running setup.py bdist_wheel for geographiclib: finished with status 'done'
  Stored in directory: C:\Users\Asus\AppData\Local\pip\Cache\wheels\99\45\d1\14954797e2a976083182c2e7da9b4e924509e59b6e5c661061
Successfully built geographiclib


You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [3]:
!pip install folium
import pandas as pd
import folium

  Using cached https://files.pythonhosted.org/packages/55/e2/7e523df8558b7f4b2ab4c62014fd378ccecce3fdc14c9928b272a88ae4cc/folium-0.7.0-py3-none-any.whl
  Using cached https://files.pythonhosted.org/packages/63/36/1c93318e9653f4e414a2e0c3b98fc898b4970e939afeedeee6075dd3b703/branca-0.3.1-py3-none-any.whl


You are using pip version 9.0.1, however version 18.1 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.


In [69]:
data = pd.read_csv('Airbnb_Texas_Rentals.csv', usecols= ['latitude','longitude','title'])
data=data.dropna()
data=data.reset_index(drop=True)

In [101]:
distanceFrom=[float(input()),float(input())]

32.862116
-97.002643


In [102]:
dis=float(input())

20


In [122]:
m = folium.Map(location=[distanceFrom[0],distanceFrom[1]])

folium.Circle(
    location=[distanceFrom[0],distanceFrom[1]],
    radius=dis*1000,
    color='#3186cc',
    fill=True,
    fill_color='#3186cc'
).add_to(m)

for i in range(len(data)):
    coor=(data['latitude'][i], data['longitude'][i])
    if distance.distance(distanceFrom, coor).km<=dis:
        folium.Marker(coor, popup=f"{data['title'][i]}").add_to(m)

m